In [1]:
import urllib.request
import gc

In [2]:
from sklearn.cluster import AgglomerativeClustering

In [80]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
import matplotlib
import matplotlib.pyplot as plt
from sklearn import datasets
from scipy import stats
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import datetime
import re
import plotly.figure_factory as ff
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt

In [5]:
from sklearn.cluster import KMeans

In [6]:
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_rows', 300) 

In [7]:
hist_data_full=pd.read_csv("hist_data.csv")

In [8]:
test = pd.read_csv("test.csv")

In [9]:
pop_goods_sorted = pd.read_excel("num_item_fin.xlsx")

In [10]:
# Выбираем топ 50 продуктов
pop_goods_sorted_top = pop_goods_sorted.head(50)
pop_goods_sorted_top

,item_id,pav_order_id,cs,ps
0,202820148.0,65840,0.0,0.0000
1,202872237.0,41658,41658.0,0.0103
2,202809628.0,27245,68903.0,0.0171
3,202838754.0,17863,86766.0,0.0215
4,203059303.0,17378,104144.0,0.0258
5,203404725.0,17099,121243.0,0.0300
6,202854572.0,16188,137431.0,0.0340
7,202791192.0,15284,152715.0,0.0378
8,202864758.0,15183,167898.0,0.0416
9,203068900.0,15109,183007.0,0.0453


In [11]:
pop_goods_sorted_top_list = list(pop_goods_sorted_top['item_id'].unique())
pop_goods_sorted_top_list

[202820148.0,
 202872237.0,
 202809628.0,
 202838754.0,
 203059303.0,
 203404725.0,
 202854572.0,
 202791192.0,
 202864758.0,
 203068900.0,
 202807362.0,
 202953901.0,
 212519969.0,
 202880262.0,
 202791620.0,
 202820144.0,
 203370019.0,
 202820288.0,
 202820143.0,
 203566372.0,
 202880254.0,
 202882452.0,
 203434856.0,
 202963924.0,
 203566371.0,
 202791611.0,
 202876533.0,
 203405624.0,
 203087846.0,
 203403088.0,
 202936013.0,
 203403030.0,
 202822135.0,
 203403181.0,
 202819114.0,
 203549854.0,
 202910396.0,
 202808298.0,
 202967706.0,
 203367249.0,
 202808824.0,
 203442947.0,
 202822471.0,
 203118216.0,
 203367251.0,
 202920228.0,
 203087853.0,
 202803557.0,
 212520067.0]

In [12]:
# Делаем рандомную выборку меньшего размера при необходимости
# hist_data = hist_data_full.sample(n = 30000)
# hist_data = hist_data_full.sample(frac =.02)

In [13]:
hist_data = hist_data_full

In [14]:
hist_data

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14
...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,2021-09-07 18:48:29,204351817,1.000,99.990,False,9.84
4529885,95619205,4620221347,2021-09-07 18:48:29,203523604,1.000,62.190,False,9.84
4529886,95619205,4620221347,2021-09-07 18:48:29,203566357,2.000,39.890,False,9.84
4529887,95619205,4620221347,2021-09-07 18:48:29,203566418,1.000,62.190,False,9.84


In [15]:
hist_data['created_date'] = pd.to_datetime(hist_data['created']).dt.date

In [16]:
hist_data['created_time'] = pd.to_datetime(hist_data['created']).dt.time

In [17]:
hist_data['price_per_unit'] = hist_data['price_sold']/hist_data['count']

In [18]:
# hist_data_new = hist_data[(hist_data['item_id'] in pop_goods_sorted_top5_list)] НЕ РАБОТАЕТ

In [19]:
pop_goods_sorted_top_list

[202820148.0,
 202872237.0,
 202809628.0,
 202838754.0,
 203059303.0,
 203404725.0,
 202854572.0,
 202791192.0,
 202864758.0,
 203068900.0,
 202807362.0,
 202953901.0,
 212519969.0,
 202880262.0,
 202791620.0,
 202820144.0,
 203370019.0,
 202820288.0,
 202820143.0,
 203566372.0,
 202880254.0,
 202882452.0,
 203434856.0,
 202963924.0,
 203566371.0,
 202791611.0,
 202876533.0,
 203405624.0,
 203087846.0,
 203403088.0,
 202936013.0,
 203403030.0,
 202822135.0,
 203403181.0,
 202819114.0,
 203549854.0,
 202910396.0,
 202808298.0,
 202967706.0,
 203367249.0,
 202808824.0,
 203442947.0,
 202822471.0,
 203118216.0,
 203367251.0,
 202920228.0,
 203087853.0,
 202803557.0,
 212520067.0]

In [20]:
hist_data_topgoods = hist_data.loc[hist_data['item_id'].isin(pop_goods_sorted_top_list)]

In [21]:
hist_data_topgoods

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight,created_date,created_time,price_per_unit
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14,2021-07-01,00:03:44,20.940669
5,95203091,98506637863,2021-07-01 00:03:44,202936013,1.000,109.890,False,11.14,2021-07-01,00:03:44,109.890000
6,95203091,98506637863,2021-07-01 00:03:44,202838754,1.002,43.945,True,11.14,2021-07-01,00:03:44,43.857285
7,95203091,98506637863,2021-07-01 00:03:44,203068900,0.968,52.395,True,11.14,2021-07-01,00:03:44,54.127066
8,95203091,98506637863,2021-07-01 00:03:44,202880254,0.958,95.945,True,11.14,2021-07-01,00:03:44,100.151357
...,...,...,...,...,...,...,...,...,...,...,...
4529745,95618806,4620221031,2021-09-07 14:39:11,203566371,12.000,59.890,False,44.52,2021-09-07,14:39:11,4.990833
4529761,95618806,4620221069,2021-09-07 15:06:17,212519969,12.000,54.990,False,24.41,2021-09-07,15:06:17,4.582500
4529777,94909652,4620221155,2021-09-07 16:00:47,203403030,1.020,117.500,True,3.98,2021-09-07,16:00:47,115.196078
4529804,94923960,98521217318,2021-09-07 16:17:36,202791192,2.000,59.890,False,15.03,2021-09-07,16:17:36,29.945000


### Определяем территории и принедлежность пользователей к территориям

In [22]:
# Определяем 




In [23]:
hist_data.shape[0]

4529889

In [24]:
#пилим на кусочки
chunk_size = 5000
chunks = [x for x in range(0, hist_data.shape[0], chunk_size)]
for i in range(0, len(chunks) - 1):
    print(chunks[i], chunks[i + 1] - 1)

0 4999
5000 9999
10000 14999
15000 19999
20000 24999
25000 29999
30000 34999
35000 39999
40000 44999
45000 49999
50000 54999
55000 59999
60000 64999
65000 69999
70000 74999
75000 79999
80000 84999
85000 89999
90000 94999
95000 99999
100000 104999
105000 109999
110000 114999
115000 119999
120000 124999
125000 129999
130000 134999
135000 139999
140000 144999
145000 149999
150000 154999
155000 159999
160000 164999
165000 169999
170000 174999
175000 179999
180000 184999
185000 189999
190000 194999
195000 199999
200000 204999
205000 209999
210000 214999
215000 219999
220000 224999
225000 229999
230000 234999
235000 239999
240000 244999
245000 249999
250000 254999
255000 259999
260000 264999
265000 269999
270000 274999
275000 279999
280000 284999
285000 289999
290000 294999
295000 299999
300000 304999
305000 309999
310000 314999
315000 319999
320000 324999
325000 329999
330000 334999
335000 339999
340000 344999
345000 349999
350000 354999
355000 359999
360000 364999
365000 369999
370000 3749

In [25]:
#пилим на кусочки
chunk_size = 5000
chunks = [x for x in range(0, hist_data.shape[0], chunk_size)]
for i in range(0, len(chunks) - 1):
    print(chunks[i], chunks[i + 1] - 1)
%%time
1 + 1
df_new = pd.concat([hist_data_topgoods.iloc[ chunks[i]:chunks[i + 1] - 1 ].pivot_table(index='buyer_id', columns='item_id', values='price_sold') for i in range(0, len(chunks) - 1)])

0 4999
5000 9999
10000 14999
15000 19999
20000 24999
25000 29999
30000 34999
35000 39999
40000 44999
45000 49999
50000 54999
55000 59999
60000 64999
65000 69999
70000 74999
75000 79999
80000 84999
85000 89999
90000 94999
95000 99999
100000 104999
105000 109999
110000 114999
115000 119999
120000 124999
125000 129999
130000 134999
135000 139999
140000 144999
145000 149999
150000 154999
155000 159999
160000 164999
165000 169999
170000 174999
175000 179999
180000 184999
185000 189999
190000 194999
195000 199999
200000 204999
205000 209999
210000 214999
215000 219999
220000 224999
225000 229999
230000 234999
235000 239999
240000 244999
245000 249999
250000 254999
255000 259999
260000 264999
265000 269999
270000 274999
275000 279999
280000 284999
285000 289999
290000 294999
295000 299999
300000 304999
305000 309999
310000 314999
315000 319999
320000 324999
325000 329999
330000 334999
335000 339999
340000 344999
345000 349999
350000 354999
355000 359999
360000 364999
365000 369999
370000 3749

3820000 3824999
3825000 3829999
3830000 3834999
3835000 3839999
3840000 3844999
3845000 3849999
3850000 3854999
3855000 3859999
3860000 3864999
3865000 3869999
3870000 3874999
3875000 3879999
3880000 3884999
3885000 3889999
3890000 3894999
3895000 3899999
3900000 3904999
3905000 3909999
3910000 3914999
3915000 3919999
3920000 3924999
3925000 3929999
3930000 3934999
3935000 3939999
3940000 3944999
3945000 3949999
3950000 3954999
3955000 3959999
3960000 3964999
3965000 3969999
3970000 3974999
3975000 3979999
3980000 3984999
3985000 3989999
3990000 3994999
3995000 3999999
4000000 4004999
4005000 4009999
4010000 4014999
4015000 4019999
4020000 4024999
4025000 4029999
4030000 4034999
4035000 4039999
4040000 4044999
4045000 4049999
4050000 4054999
4055000 4059999
4060000 4064999
4065000 4069999
4070000 4074999
4075000 4079999
4080000 4084999
4085000 4089999
4090000 4094999
4095000 4099999
4100000 4104999
4105000 4109999
4110000 4114999
4115000 4119999
4120000 4124999
4125000 4129999
4130000 

UsageError: Line magic function `%%time` not found.


In [26]:
df_new

NameError: name 'df_new' is not defined

In [27]:
%%time
1 + 1
user_good_pivot = pd.pivot_table(hist_data_topgoods, index = 'buyer_id', columns = 'item_id', values = 'price_sold', aggfunc= 'mean')

CPU times: user 279 ms, sys: 111 ms, total: 390 ms
Wall time: 439 ms


In [29]:
user_good_pivot

item_id,202791192,202791611,202791620,202803557,202807362,202808298,202808824,202809628,202819114,202820143,202820144,202820148,202820288,202822135,202822471,202838754,202854572,202864758,202872237,202876533,202880254,202880262,202882452,202910396,202920228,202936013,202953901,202963924,202967706,203059303,203068900,203087846,203087853,203118216,203367249,203367251,203370019,203403030,203403088,203403181,203404725,203405624,203434856,203442947,203549854,203566371,203566372,212519969,212520067
buyer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94578442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.99,NaN,NaN,71.790000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.22,NaN,NaN,37.995,34.995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94578475,51.55,NaN,56.99,NaN,NaN,NaN,65.74,NaN,NaN,NaN,NaN,56.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.945,74.945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94578533,NaN,NaN,NaN,NaN,44.995,NaN,NaN,11.895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.995,38.745,14.870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.995,NaN,NaN,NaN,54.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94578595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.995,13.945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,228.9,64.995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94578656,67.44,NaN,56.99,NaN,NaN,NaN,69.99,13.845,28.69,NaN,19.995,54.556667,NaN,NaN,NaN,38.195,NaN,NaN,16.945,NaN,NaN,NaN,NaN,46.628333,NaN,NaN,NaN,NaN,NaN,44.995,36.745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.99,39.995,NaN,NaN,NaN,NaN,NaN,54.89,NaN,69.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96393849,51.20,64.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.65,NaN,NaN,NaN
96394254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96395136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.690000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## K-means

In [175]:
# расстояние между двумя точками
def distance_between_points(point1, point2):
    distance = 0
    for i, coord in enumerate(point1):
        if np.isnan(coord):
            pass
        else:
            distance += (coord-point2[i])**2
        distance = distance**(1/2)
    return distance

In [176]:
def k_means_custom(k, n_iterations=10):

    # рандомно определяем центры для k кластеров
    centers_of_clusters = {}
    for i in range(k):
        center = np.random.randint(np.min(user_good_pivot.describe().iloc[3]),
                              np.max(user_good_pivot.describe().iloc[7]), size=user_good_pivot.shape[1])
        centers_of_clusters[i] = center

    
    for iteration in tqdm(range(n_iterations)):
        
        points_of_clusters = {i: [] for i in centers_of_clusters.keys()} 

        # для каждого пользователя рассчитываем минимальное расстояние до центра кластера и определяем принадлежность к кластеру
        for ind in user_good_pivot.index:
            curr_user = user_good_pivot.loc[ind]
            min_distance = float('inf')
            for i, center in centers_of_clusters.items():
                curr_distance = distance_between_points(curr_user, center)
                if curr_distance < min_distance:
                    min_distance = curr_distance
                    cluster = i
            points_of_clusters[cluster].append(curr_user)

        # обновляем центры кластеров
        # для каждого кластера
        norms = []
        for i, points in points_of_clusters.items():
            new_center = []
            # для каждого из n измерений (продуктов)
            for j, item in enumerate(user_good_pivot.columns):
                new_center_point = 0
                q = 0
                for v in points_of_clusters[i][j]:
                    if np.isnan(v):
                        pass
                    else:
                        new_center_point += v
                        q += 1
                new_center_point /= q
                new_center.append(new_center_point)
            norms.append(np.linalg.norm(np.array(centers_of_clusters[i]) - np.array(new_center)))
            centers_of_clusters[i] = new_center
        if np.mean(norms) < 0.0001:
            print(np.mean(norms))
            print('Centers dont change')
            break
    if iteration >= n_iterations-1:
        print('Iteration limit has reached')
    return points_of_clusters            

In [178]:
k = 7 # количество кластеров
n_iterations = 100 # число итераций

points_of_clusters = k_means_custom(k, n_iterations)

100%|██████████| 100/100 [45:25<00:00, 27.26s/it]

Iteration limit has reached


In [179]:
clusters = []
users = []

# маппинг пользователь-кластер
for cluster in points_of_clusters.keys():
    for i, user in enumerate(points_of_clusters[cluster]):
        clusters.append(cluster)
        users.append(user.name)

In [180]:
user_good_pivot.join(pd.DataFrame(clusters, index=users))

,202791192,202791611,202791620,202803557,202807362,202808298,202808824,202809628,202819114,202820143,202820144,202820148,202820288,202822135,202822471,202838754,202854572,202864758,202872237,202876533,202880254,202880262,202882452,202910396,202920228,202936013,202953901,202963924,202967706,203059303,203068900,203087846,203087853,203118216,203367249,203367251,203370019,203403030,203403088,203403181,203404725,203405624,203434856,203442947,203549854,203566371,203566372,212519969,212520067,0
buyer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94578442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.99,NaN,NaN,71.790000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.22,NaN,NaN,37.995,34.995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
94578475,51.55,NaN,56.99,NaN,NaN,NaN,65.74,NaN,NaN,NaN,NaN,56.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.945,74.945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
94578533,NaN,NaN,NaN,NaN,44.995,NaN,NaN,11.895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.995,38.745,14.870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.995,NaN,NaN,NaN,54.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
94578595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.995,13.945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,228.9,64.995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
94578656,67.44,NaN,56.99,NaN,NaN,NaN,69.99,13.845,28.69,NaN,19.995,54.556667,NaN,NaN,NaN,38.195,NaN,NaN,16.945,NaN,NaN,NaN,NaN,46.628333,NaN,NaN,NaN,NaN,NaN,44.995,36.745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.99,39.995,NaN,NaN,NaN,NaN,NaN,54.89,NaN,69.89,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96393849,51.20,64.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.65,NaN,NaN,NaN,1
96394254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
96395136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.690000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
